<a href="https://colab.research.google.com/github/mustafa-vu/dmpm-qb/blob/main/Statistical_and_Social_Analysis_of_Reddit_Comments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Statistical and Social Analysis of Reddit Comments

###### Here we are trying to primarily fetch the Reddit comments data that is stored in Google Cloud Platform, Big Query with the help of this python script which makes use of SQL queries to query the data to produce the required data. 


In [58]:
from google.cloud import bigquery
from google.oauth2 import service_account

cred = service_account.Credentials.from_service_account_file('project_key.json',
                                                            scopes = ["https://www.googleapis.com/auth/cloud-platform"]
                                                            )

client = bigquery.Client(
    credentials=cred,
    project=cred.project_id,
)

query = """
SELECT * FROM `fh-bigquery.reddit_comments.2015_01` LIMIT 10
"""

query_job=client.query(query)
data=query_job.result()

for row in data:
    print(row, end = '\n\n')

Row(('Die meisten würden sich wohl nicht trauen, weil dann das Karma echt schnell weg geht bei der politischen Grundhaltung hier in /r/de.\n\nWer das Positionspapier von Pegida (über den Punkt von christlich-jüdischem Abendland kann man gerne streiten) nicht akzeptiert, stellt sich gegen die Grundsätze unserer Demokratie. Dass es eine Protestbewegung gibt, die sich für die Durchsetzung bestehender Gesetze einsetzt, ist schon erstaunlich genug.\n\nEhemalige Schulkamaraden, die jetzt in Dresden studieren, habe ich mal über die Weihnachtsferien ausgefragt, was da so für Leute rumrennen. Und nach deren Einschätzung, sind das politisch desillusionierte Leute, die aber genau wissen, was in der Welt vor sich geht (natürlich nicht alle). Sie wissen einiges über die Weltpolitik der letzten 30-40 Jahre und haben eine gewisse Ahnung von Geopolitik, zumindest mehr als man in Deutschland erfährt, wenn man sich nur berieseln lässt.\n\nViele Rentner waren auch schon 89 auf der Straße und verspüren ei

    
**Here this data shows majorly the comment body ( which is the comment posted by user), the Author name (the user who posted this comment ), timestamp of creation in UTC, subreddit ( which is like the domain or scope of a comment), score ( which is simply the number of upvotes minus the number of downvotes. If five users like the comment and three users don't it will have a score of 2).**


## Business Outcomes

### 1. Analyzing trends based on comment data from 2015 - 2019.





###### Here we can modify the above query to get the desired result that we will need for analysis with the help of regular expressions.

    We have included major technical and trending Subreddits which will help us to refine our query in a way to give the data which will be helpful to analyze the trending topics or products.
    
    Also, we have designed the query in a way that it excludes all the major stopwords that may occur in the Reddit comments.

    Instead of displaying all the data repeatedly, we are now storing this python script output in a CSV file which we can later access for further analysis. Here we are running this script on the dataset of a random month of a year as an example.



In [59]:


from google.cloud import bigquery
from google.oauth2 import service_account

import csv


cred = service_account.Credentials.from_service_account_file('project_key.json',
                                                            scopes = ["https://www.googleapis.com/auth/cloud-platform"]
                                                            )

client = bigquery.Client(
    credentials=cred,
    project=cred.project_id,
)


query = """
SELECT word, COUNT(*) c FROM (
  SELECT REGEXP_EXTRACT_ALL(LOWER(body), r'[a-z][a-z][a-z]+[a-z]?') words
  FROM `fh-bigquery.reddit_comments.2017_11`
  WHERE score>10 AND subreddit IN ('technology', 'tech','realtech','Computing','Windows','Apple','Mac','Linux',\
  'Ubuntu','Chromium','WinTiles','windowsazure','Mobile','Android','ios','iphone','Blackberry','WindowsPhone',\
  'Windows','symbian','UbuntuPhone','meego','nokia','ipad','Surface','gaming','Games','pcgaming','Steam','SteamBox',\
  'lanparty','LowEndGaming','PatientGamers','xbox','xoxone','xbox360','xboxLive','playstation','PS4','Vita',\
  'VitaTV','PS3','PlayStationPlus','nintendo','wii','wiiu','3ds','nds','iosgaming','Gadgets','ipod','Zune',\
  'GoogleGlass','startups','bitcoin','BitcoinMarkets','CryptoMarkets','Kickstarter','webmarketing','seo','bigseo',\
  'dotcom','Information Assurance','techsupport','24hourssupport','Applehelp','linux4noobs','pcgamingtechsupport',\
  'asktechnology','techolitics','netsec','netneutrality','talesfromtechsupport','technologymeta','CSEducation',\
  'cscareerquestions','GameDealsMeta','computertechs','Sysadmin','Hardware','Hardwarenews','buildapc','networking',\
  'intel','nvidia','amd','datacenter','monitors','mechanicalkeyboards','MouseReview','TrackBalls','freewareindex',\
  'CAD','AutoCAD','programming','learnprogramming','redditdev','coding','compsci','Software','dailyprogrammer',\
  'dailyscripts','webdev','gamedev','webdev','web_design','userexperience','firefox','Chrome','internetexplorer',\
  'TechnologyPorn','battlestations','techsupportgore','softwaregore','techsupportmacgyver','ImaginaryTechnology',\
  'buildapcsales','suggestalaptop','gamedeals','gameoffers','AppHookup','SteamGameSwap','ShouldIbuythisgame',\
  'Futurology','RenewableEnergy','Engineering','Geek','airz23', 'python','javascript', 'java', 'csharp', 'cpp',\
  'php', 'rust', 'ruby', 'swift', 'sql', 'scala', 'matlab', 'kotlin', 'latex', 'dartlang', 'perl', 'androidapps',\
  'AndroidWear', 'buildapc', 'microsoft', 'oculus', 'virtualreality', 'AppleEnthusiasts', 'windowsinsiders',\
  'AppleMusic', 'macsetups', 'Geek', 'networking', 'VPN', 'vpnreviews', 'VPNTorrents', 'Netflix', 'NetfixviaVPN',\
  'Gadgets')
), UNNEST(words) word
WHERE word NOT IN (SELECT word FROM(
  SELECT word, COUNT(*) c FROM (
    SELECT REGEXP_EXTRACT_ALL(LOWER(body), r'[a-z][a-z][a-z]+[a-z]?') words
    FROM `fh-bigquery.reddit_comments.2017_11`
    WHERE score>10
    
  ), UNNEST(words) word
  GROUP BY word
  ORDER BY c DESC
  LIMIT 300
))
GROUP BY word
ORDER BY c DESC 
LIMIT 100

"""

query_job=client.query(query)
data=query_job.result()

#for row in data:
 #   print(row)
    
# Create the csv file
with open('testdata.csv', 'w', newline='') as f_handle:
    writer = csv.writer(f_handle)
    # Add the header/column names
    header = ['word', 'count']
    writer.writerow(header)
    # Iterate over `data`  and  write to the csv file
    for row in data:
        writer.writerow(row)

###### Similarly, we can store the output for dataset of all the months from year 2015 - 2019 and then combine all the files to make a final file of all the results “final_results.csv”.


    A glimpse of how final_results.csv look like.

**Now to analyze the trends properly in this data that we have retrieved from our python script and SQL query, we need to visualize and represent it properly.**

For this purpose, we are using **line plot**.
    
    Line plot is good at showing specific values of data, meaning that given one variable the other can easily be determined. They show trends in data clearly, meaning that they visibly show how one variable is affected by the other as it increases or decreases.


In [61]:
import pandas as pd
import plotly.express as px
#df = pd.read_csv('final_results.csv')
df = pd.read_csv('testdata.csv')

fig = px.line(df, x = 'word', y = 'count', title='All major Trends on bigquery from 2015-2019', template = 'ggplot2')
fig.show()

**Hence it shows clearly how the trend proceeds.**

### 2. Top trending features of a given product

#### 2.1 : Now we have written a script that will analyze the data based on a particular product or topic (given as a keyword) and find top trending features of that product.

    Here we are considering top 10 features and taken product ( keyword ) as oneplus to analyze its trends.
 

In [128]:

query = """
SELECT word, COUNT(*) c FROM (
  SELECT REGEXP_EXTRACT_ALL(LOWER(body), r'[a-z][a-z][a-z]+[a-z]?') words
  FROM `fh-bigquery.reddit_comments.2017_11`
  WHERE score>10 AND subreddit='oneplus'
), UNNEST(words) word
WHERE word NOT IN (SELECT word FROM(
  SELECT word, COUNT(*) c FROM (
    SELECT REGEXP_EXTRACT_ALL(LOWER(body), r'[a-z][a-z][a-z]+[a-z]?') words
    FROM `fh-bigquery.reddit_comments.2017_11`
    WHERE score>10 AND subreddit NOT IN ('politics', 'funny')
    
  ), UNNEST(words) word
  GROUP BY word
  ORDER BY c DESC
  LIMIT 300
))
GROUP BY word
ORDER BY c DESC 
LIMIT 100


"""

query_job=client.query(query)
data=query_job.result()

# Create the csv file
with open('oneplus.csv', 'w', newline='') as f_handle:
    writer = csv.writer(f_handle)
    # Add the header/column names
    header = ['word', 'c']
    writer.writerow(header)
    # Iterate over `data`  and  write to the csv file
    for ro in data:
        writer.writerow(ro)

In [129]:
df = pd.read_csv('oneplus.csv')

fig = px.bar(df, x = 'word', y = 'c', title='Topn 10 Trends on bigquery from of a particular product.', template = 'plotly_white')
fig.show()

#### 2.2 : Now we have modified the above script that will analyze the data based on a particular product or topic (given as a keyword) and find trending features of that product based on a given range.

In [65]:
#iterate over frequency
import pandas as pd


from google.cloud import bigquery
from google.oauth2 import service_account

import csv


cred = service_account.Credentials.from_service_account_file('project_key.json',
                                                            scopes = ["https://www.googleapis.com/auth/cloud-platform"]
                                                            )

client = bigquery.Client(
    credentials=cred,
    project=cred.project_id,
)


query = """
SELECT word, COUNT(*) c FROM (
  SELECT REGEXP_EXTRACT_ALL(LOWER(body), r'[a-z][a-z][a-z]+[a-z]?') words
  FROM `fh-bigquery.reddit_comments.2017_11`
  WHERE score>10 AND subreddit='oneplus'
), UNNEST(words) word
WHERE word NOT IN (SELECT word FROM(
  SELECT word, COUNT(*) c FROM (
    SELECT REGEXP_EXTRACT_ALL(LOWER(body), r'[a-z][a-z][a-z]+[a-z]?') words
    FROM `fh-bigquery.reddit_comments.2017_11`
    WHERE score>10 AND subreddit NOT IN ('politics', 'funny')
    
  ), UNNEST(words) word
  GROUP BY word
  ORDER BY c DESC
  LIMIT 300
))
GROUP BY word
ORDER BY c DESC 


"""

query_job=client.query(query)
data=query_job.result()

df = pd.read_csv('oneplus.csv')
saved_column = df['c']
#print(saved_column[0])

print(f"Input the range of data to be accessed based on frequency.\nAt max it can be {saved_column[0]}\n example: for frquency range 20 to 50 lower_range =20 and will have a higher frequecy data and upper_range = 50")

lr = int(input("Lower Range : "))
ur = int(input('Upper Range : '))
sep =','
#for row in open("oneplus3.csv"):

with open('oneplus_range.csv', 'w', newline='') as f_handle:
    writer = csv.writer(f_handle)
    header = ['word', 'c']
    writer.writerow(header)
    for row, r in zip(saved_column, data):
        #print(row)
        #print(type(row))
        if row<=ur and row>=lr:
            writer.writerow(r)

Input the range of data to be accessed based on frequency.
At max it can be 431
 example: for frquency range 20 to 50 lower_range =20 and will have a higher frequecy data and upper_range = 50
Lower Range : 10
Upper Range : 431


In [67]:
import pandas as pd
import plotly.express as px
df = pd.read_csv('oneplus_range.csv')

fig = px.histogram(df, x = 'word', y = 'c', title=f'Trends on bigquery of a particular product within range {lr} - {ur}.', template = 'plotly_white', color_discrete_sequence=['darkblue'])
fig.show()

### 3. Analyze a less reviewd product


**From the above visualization we picked wikipedia to perform further analysis.**

**Analyzing wikipedia as a producyt with comparatively less reviews (from wikipedia subreddit)**

In [91]:

query = """
SELECT word, COUNT(*) c FROM (
  SELECT REGEXP_EXTRACT_ALL(LOWER(body), r'[a-z][a-z][a-z]+[a-z]?') words
  FROM `fh-bigquery.reddit_comments.2019_08`
  WHERE score>10 AND subreddit='wikipedia'
), UNNEST(words) word
WHERE word NOT IN (SELECT word FROM(
  SELECT word, COUNT(*) c FROM (
    SELECT REGEXP_EXTRACT_ALL(LOWER(body), r'[a-z][a-z][a-z]+[a-z]?') words
    FROM `fh-bigquery.reddit_comments.2019_08`
    WHERE score>10
    
  ), UNNEST(words) word
  GROUP BY word
  ORDER BY c DESC
  LIMIT 300
))
GROUP BY word
ORDER BY c DESC 

"""

query_job=client.query(query)
data=query_job.result()

# Create the csv file
with open('wikipedia.csv', 'w', newline='') as f_handle:
    writer = csv.writer(f_handle)
    # Add the header/column names
    header = ['word', 'c']
    writer.writerow(header)
    # Iterate over `data`  and  write to the csv file
    for ro in data:
        writer.writerow(ro)

In [92]:
!type wikipedia.csv

/bin/bash: line 0: type: wikipedia.csv: not found


In [94]:
df = pd.read_csv('wikipedia.csv')

fig = px.histogram(df, x = 'word', y = 'c', title=f'Trends on bigquery about wikipedia', template = 'plotly_white', color_discrete_sequence=['chocolate'])
fig.show()

**Now extracting sentiments from above analysis**

In [95]:
import pandas as pd            
from textblob import TextBlob  
from itertools import islice   
def to_lower(data):
    return data.lower()

In [96]:
df_survey_data = pd.read_csv("wikipedia.csv")



In [97]:
COLS = ['text', 'sentiment','subjectivity','polarity']

In [98]:
df = pd.DataFrame(columns=COLS)


In [99]:
df_survey_data.columns

Index(['word', 'c'], dtype='object')

In [100]:
for index, row in islice(df_survey_data.iterrows(), 0, None):
    new_entry = []
    text_lower = to_lower(row['word'])
    blob = TextBlob(text_lower)
    sentiment = blob.sentiment
    polarity = sentiment.polarity
    subjectivity = sentiment.subjectivity
    new_entry += [text_lower,sentiment,subjectivity,polarity]    
    single_survey_sentimet_df = pd.DataFrame([new_entry], columns=COLS)
    df = df.append(single_survey_sentimet_df, ignore_index=True, sort=True)


In [101]:
df.to_csv('sentiments_lower.csv', mode='w', columns=COLS, index=False, encoding='utf-8')

In [102]:
df.head()

,polarity,sentiment,subjectivity,text
0,0.0,"(0.0, 0.0)",0.0,wikipedia
1,0.0,"(0.0, 0.0)",0.0,org
2,0.0,"(0.0, 0.0)",0.0,wiki
3,0.0,"(0.0, 0.0)",0.0,war
4,0.0,"(0.0, 0.0)",0.0,french


In [103]:
df[df['subjectivity']!=0].head(20) # filtering the neutral comments

,polarity,sentiment,subjectivity,text
18,0.500000,"(0.5, 0.5)",0.500000,interesting
26,0.000000,"(0.0, 0.4)",0.400000,related
30,-0.300000,"(-0.3, 0.5)",0.500000,common
57,0.000000,"(0.0, 0.4)",0.400000,similar
70,-0.200000,"(-0.2, 0.4)",0.400000,dead
73,0.100000,"(0.1, 0.4)",0.400000,young
76,0.000000,"(0.0, 0.05)",0.050000,outside
80,0.400000,"(0.4, 1.0)",1.000000,important
81,0.200000,"(0.2, 0.2)",0.200000,thanks
108,0.000000,"(0.0, 1.0)",1.000000,likely


In [104]:
df.describe().round(3)

,polarity,subjectivity
count,3245.000,3245.000
mean,0.003,0.064
std,0.135,0.212
min,-1.000,0.000
25%,0.000,0.000
50%,0.000,0.000
75%,0.000,0.000
max,1.000,1.000


In [105]:
df2 = pd.read_csv('sentiments_lower.csv')

fig2 = px.violin(df2, x = 'subjectivity', y = 'polarity', title='Plot Between subjectivity and polarity', template = 'plotly')
fig2.show()

In [106]:
dffilter2 = df.loc[(df.loc[:, df.dtypes != object] == 0).any(1)]
dffilter2.to_csv('sentiments_lower_filter.csv', mode='w', columns=COLS, index=False, encoding='utf-8')
 
dfp2 = df.loc[(df.loc[:, df.dtypes != object] > 0).any(1)]
dfp2.to_csv('sentiments_lower_pos.csv', mode='w', columns=COLS, index=False, encoding='utf-8')
 
dfn2 = df.loc[(df2.loc[:, df.dtypes != object] < 0).any(1)]
dfn2.to_csv('sentiments_lower_neg.csv', mode='w', columns=COLS, index=False, encoding='utf-8')
 
rcn2=0
rcp2=0
rcf2=0
rcall2=0
for row in open("sentiments_lower_pos.csv"):
    rcp2+=1
for row in open("sentiments_lower_neg.csv"):
    rcn2+=1
for row in open("sentiments_lower_filter.csv"):
    rcf2+=1
for row in open("sentiments_lower.csv"):
    rcall2+=1
 
print(f"\n\npositive remarks = {rcp2}, negative remarks = {rcn2}, and neutral remarks = {rcf2}\n\nRelation of neutral \
remarks with repect to All remarks is {round((rcf2/rcall2),3)}\n\nRelation of positive remarks with repect to neutral \
is {round((rcp2/rcf2),3)}\n\nRelation of negative remarks with repect to neutral is {round((rcn2/rcf2),3)}\n\n")




positive remarks = 348, negative remarks = 120, and neutral remarks = 2978

Relation of neutral remarks with repect to All remarks is 0.917

Relation of positive remarks with repect to neutral is 0.117

Relation of negative remarks with repect to neutral is 0.04




## 5. Now comparing the results of wikipedia analysis with oneplus analysis

**Results of oneplus anaysis:**

In [107]:
df.describe().round(3)

,polarity,subjectivity
count,3245.000,3245.000
mean,0.003,0.064
std,0.135,0.212
min,-1.000,0.000
25%,0.000,0.000
50%,0.000,0.000
75%,0.000,0.000
max,1.000,1.000


In [108]:
print(f"\n\npositive remarks = {rcp}, negetive remarks = {rcn}, and neutral remarks = {rcf}\n\nRelation of neutral\
remarks with repect to All remarks is {round((rcf/rcall),3)}\n\nRelation of positive remarks with repect to neutral\
is {round((rcp/rcf),3)}\n\nRelation of negetive remarks with repect to neutral is {round((rcn/rcf),3)}\n\n")



positive remarks = 1, negetive remarks = 1, and neutral remarks = 11

Relation of neutralremarks with repect to All remarks is 1.0

Relation of positive remarks with repect to neutralis 0.091

Relation of negetive remarks with repect to neutral is 0.091




**Results of wikipedia analysis:**

In [109]:
df.describe().round(3)

,polarity,subjectivity
count,3245.000,3245.000
mean,0.003,0.064
std,0.135,0.212
min,-1.000,0.000
25%,0.000,0.000
50%,0.000,0.000
75%,0.000,0.000
max,1.000,1.000


In [110]:
print(f"\n\npositive remarks = {rcp2}, negative remarks = {rcn2}, and neutral remarks = {rcf2}\n\nRelation of neutral \
remarks with repect to All remarks is {round((rcf2/rcall2),3)}\n\nRelation of positive remarks with repect to neutral \
is {round((rcp2/rcf2),3)}\n\nRelation of negative remarks with repect to neutral is {round((rcn2/rcf2),3)}\n\n")




positive remarks = 348, negative remarks = 120, and neutral remarks = 2978

Relation of neutral remarks with repect to All remarks is 0.917

Relation of positive remarks with repect to neutral is 0.117

Relation of negative remarks with repect to neutral is 0.04




    Here we can observe that the weightage of neutral comments with respect to all remarks have slightly increased. Wikipedia being another community site where anyone can post any article, hence justifies its more neutral remarks.
    
    As the portion of neutral remarks is more and the total number of words or remarks being less, it in turn decreases the weightage of positive and negative comments for wikipedia.
    
    Moreover the mean of subjectivity of wikipedia result is more inclined towards 0 than that of older(oneplus) data. This shows that the remarks are more factual which further justifies the presence of more neutral comments.
